In [2]:
import pandas as pd
import os
from openpyxl import load_workbook

In [3]:
# CUANDO EL USUARIO CARGA LOS ARCHIVOS DEBE SELECCIONAR DE QUÉ TRATA Y EL PROGRAMA ASIGNARÁ AUTOMÁTICAMENTE UN NOMBRE

In [4]:
# ENLISTAR ARCHIVOS DE CONTABILIDAD

dir =  r"./BD CONTABILIDAD"

In [5]:
# FUNCIÓN QUE RENOMBRA LOS ARCHIVOS

def rename_file(dir, names_list):
    carpeta = os.listdir(dir)
    for archivo in carpeta:
        for name in names_list:
            if name in archivo.lower():
                os.rename(os.path.join(dir, archivo), os.path.join(dir, f'{name}.xlsx'))

In [6]:
# CAMBIAR NOMBRE DE LOS ARCHIVOS
names_list = ['diferidos', 'intangibles', 'depreciaciones']
rename_file(dir, names_list)

In [7]:
# ENLISTAR ARCHIVOS DE CONTABILIDAD

carpeta = os.listdir(dir)
carpeta

['depreciaciones.xlsx', 'diferidos.xlsx', 'intangibles.xlsx']

In [8]:
# FUNCIÓN PARA IDENTIFICAR LA HOJA DE INTERÉS EN CADA ARCHIVO

def select_sheet(dir):
    carpeta = os.listdir(dir)
    sheet_by_file = {}
    
    for archivo in carpeta:
        wb = load_workbook(os.path.join(dir, archivo), read_only=True, data_only=True)
        hojas_visibles = [ws.title for ws in wb.worksheets if ws.sheet_state == "visible"]
        hoja_base = [x for x in hojas_visibles if x.lower() != 'td']
        sheet_by_file[archivo] = hoja_base[0] if hoja_base else None
    
    return sheet_by_file

In [9]:
hojas_archivos = select_sheet(dir)
hojas_archivos

{'depreciaciones.xlsx': 'Consolidado',
 'diferidos.xlsx': 'Diferidos',
 'intangibles.xlsx': 'Consolidado'}

In [10]:
# Necesito leer cada archivo en un dataframe usando la respectiva hoja que está en el diccionario, el dataframe debe llevar por nombre el nombre del archivo. Por ejemplo, el archivo 'depreciaciones.xlsx' con la hoja 'Consolidado' debe quedar almacenado en un dataframe llamado depreciaciones
data_frames = {}
for archivo, hoja in hojas_archivos.items():
    if hoja:
        df = pd.read_excel(os.path.join(dir, archivo), sheet_name=hoja)
        data_frames[archivo] = df

In [11]:
depreciaciones = data_frames.get('depreciaciones.xlsx')
diferidos = data_frames.get('diferidos.xlsx')
intangibles = data_frames.get('intangibles.xlsx')

In [12]:
# Función que identifica el encabezado de la tabla

def identify_header(df):
    total_index = df.apply(lambda row: row.astype(str).str.lower().str.contains('total').any(), axis=1)
    first_total_index = int(total_index[total_index].index[0])
    df.columns = df.iloc[first_total_index]
    return df.drop(df.index[:first_total_index + 1]).reset_index(drop=True)
    

In [13]:
depreciaciones = identify_header(depreciaciones)
diferidos = identify_header(diferidos)
intangibles = identify_header(intangibles)

In [14]:
# Convertir los nombres de las columnas
depreciaciones.columns = depreciaciones.columns.astype(str).str.lower().str.strip()
diferidos.columns = diferidos.columns.astype(str).str.lower().str.strip()
intangibles.columns = intangibles.columns.astype(str).str.lower().str.strip()

In [15]:
print(depreciaciones.columns)
print(diferidos.columns)
print(intangibles.columns)

Index(['activo fijo', 'subnúmero', 'act + sub', 'nota de inventario',
       'alta original el', 'fecha inicio dep', 'vida util', 'vida util meses',
       'denominación', 'val.adq.', 'valor residual', 'amo acum.', 'val.cont.',
       'clase activos fijos', 'nombre clase', 'centro de coste',
       'denominación ceco', 'grupo cuenta', 'cuenta', 'nombre cuenta',
       'gasto amort', 'fecha espe', 'meses pend x amort',
       'valor amo julio 2025 - diciembre 2025', 'julio 2025', 'agosto 2025',
       'septiembre 2025', 'octubre 2025', 'noviembre 2025', 'diciembre 2025',
       'total'],
      dtype='object', name=0)
Index(['activo fijo', 'sub', 'act+subn', 'fecha', 'fecha inicio dep',
       'denominación', 'proveedor', 'val.adq.', 'amo acum.', 'val.cont.',
       'vida util', 'vida util meses', 'clase activos fijos', 'nom clase',
       'ceco', 'nombre ceco', 'grup cta deprec', 'cuenta deprec',
       'nombre cuenta', 'gasto amort', 'fecha espe', 'meses pend x amort',
       'valor am

In [16]:
ceco_variants = ['centro de coste', 'centro de costo', 'ceco', 'ce co', 'ce_co', 'c coste', 'c costo']
cuenta_variants = ['cuenta', 'cuenta deprec', 'cta', 'cta deprec']

In [17]:
# Función que normaliza el nombre de una columna
def normalize_name_column(df, variants, name):
    columnas = list(df.columns)
    columnas_norm = {c.lower().strip(): c for c in columnas}
    
    coincidencias = [columnas_norm[c] for c in columnas_norm if c in variants]
    
    return df.rename(columns={coincidencias[0]: name})

In [18]:
# Normalizar nombre de la columna CECO
depreciaciones = normalize_name_column(depreciaciones, ceco_variants, 'ceco')
diferidos = normalize_name_column(diferidos, ceco_variants, 'ceco')
intangibles = normalize_name_column(intangibles, ceco_variants, 'ceco')

In [19]:
# Normalizar nombre de la columna CUENTA
depreciaciones = normalize_name_column(depreciaciones, cuenta_variants, 'cuenta')
diferidos = normalize_name_column(diferidos, cuenta_variants, 'cuenta')
intangibles = normalize_name_column(intangibles, cuenta_variants, 'cuenta')

In [20]:
print(depreciaciones.columns)
print(diferidos.columns)
print(intangibles.columns)

Index(['activo fijo', 'subnúmero', 'act + sub', 'nota de inventario',
       'alta original el', 'fecha inicio dep', 'vida util', 'vida util meses',
       'denominación', 'val.adq.', 'valor residual', 'amo acum.', 'val.cont.',
       'clase activos fijos', 'nombre clase', 'ceco', 'denominación ceco',
       'grupo cuenta', 'cuenta', 'nombre cuenta', 'gasto amort', 'fecha espe',
       'meses pend x amort', 'valor amo julio 2025 - diciembre 2025',
       'julio 2025', 'agosto 2025', 'septiembre 2025', 'octubre 2025',
       'noviembre 2025', 'diciembre 2025', 'total'],
      dtype='object', name=0)
Index(['activo fijo', 'sub', 'act+subn', 'fecha', 'fecha inicio dep',
       'denominación', 'proveedor', 'val.adq.', 'amo acum.', 'val.cont.',
       'vida util', 'vida util meses', 'clase activos fijos', 'nom clase',
       'ceco', 'nombre ceco', 'grup cta deprec', 'cuenta', 'nombre cuenta',
       'gasto amort', 'fecha espe', 'meses pend x amort',
       'valor amort julio a diciembre 202

In [21]:
# Eliminar las filas donde la columna CECO tenga valores nulos
depreciaciones = depreciaciones.dropna(subset=['ceco'])
diferidos = diferidos.dropna(subset=['ceco'])
intangibles = intangibles.dropna(subset=['ceco']) 

In [22]:
print(depreciaciones.shape)
print(diferidos.shape)
print(intangibles.shape)

(2512, 31)
(261, 30)
(29, 30)


In [23]:
# Eliminar la columna VALOR
depreciaciones = depreciaciones.drop(columns=depreciaciones.filter(like='valor').columns)
diferidos = diferidos.drop(columns=diferidos.filter(like='valor').columns)
intangibles = intangibles.drop(columns=intangibles.filter(like='valor').columns)

In [24]:
import re

def select_ceco_month(df):
    meses_map = {
        'enero':      ['1', '01', 'ene', 'enero'],
        'febrero':    ['2', '02', 'feb', 'febrero'],
        'marzo':      ['3', '03', 'mar', 'marzo'],
        'abril':      ['4', '04', 'abr', 'abril'],
        'mayo':       ['5', '05', 'may', 'mayo'],
        'junio':      ['6', '06', 'jun', 'junio'],
        'julio':      ['7', '07', 'jul', 'julio'],
        'agosto':     ['8', '08', 'ago', 'agosto'],
        'septiembre': ['9', '09', 'sep', 'septiembre'],
        'octubre':    ['10', 'oct', 'octubre'],
        'noviembre':  ['11', 'nov', 'noviembre'],
        'diciembre':  ['12', 'dic', 'diciembre']
    }

    renombre = {}

    for col in df.columns:
        col_norm = col.lower().strip()

        for mes, patrones in meses_map.items():
            for patron in patrones:
                if re.search(patron, col_norm):
                    renombre[col] = mes
                    break

    df = df.rename(columns=renombre)

    columnas_meses = list(meses_map.keys())
    columnas_finales = ['ceco'] + ['cuenta'] + [c for c in columnas_meses if c in df.columns]

    return df[columnas_finales]

In [25]:
depreciaciones = select_ceco_month(depreciaciones)
diferidos = select_ceco_month(diferidos)
intangibles = select_ceco_month(intangibles)

In [26]:
print(depreciaciones.shape)
print(diferidos.shape)
print(intangibles.shape)

(2512, 8)
(261, 8)
(29, 8)


In [27]:
# MODIFICAR PORQUE LA AGRUPACIÓN ES POR CUENTA Y CECO
depreciaciones = depreciaciones.groupby(['ceco','cuenta']).sum().reset_index()
diferidos = diferidos.groupby(['ceco','cuenta']).sum().reset_index()
intangibles = intangibles.groupby(['ceco','cuenta']).sum().reset_index()

In [28]:
print(depreciaciones.shape)
print(diferidos.shape)
print(intangibles.shape)

(282, 8)
(75, 8)
(16, 8)


In [29]:
# Concatenar dataframes de forma vertical
contabilidad = pd.concat([depreciaciones, diferidos, intangibles], axis=0)

In [30]:
# Agrupación por ceco
contabilidad = contabilidad.groupby(['ceco','cuenta']).sum().reset_index()

In [31]:
contabilidad

,ceco,cuenta,julio,agosto,septiembre,octubre,noviembre,diciembre
0,CO10A00000,5160350001,4519500.326,4519500.326,4519500.326,4519500.326,4519500.326,4519500.326
1,CO10A00001,5160200001,322324,322324,322324,322324,322324,322324
2,CO10A00003,5160200001,688144,688144,688144,688144,688144,688144
3,CO10A00004,5160200001,526835,526835,526835,526835,526835,526835
4,CO10A10000,5120750002,64500,64500,0,0,0,0
...,...,...,...,...,...,...,...,...
368,CO10F52000,5260200001,465253,465253,465253,465253,465253,465253
369,CO10F60100,5220750002,3026233,3026233,3026233,3026233,3026233,3026233
370,CO10F60100,5260200001,804809,804809,804809,618784,618784,618784
371,CO10G00001,5260100001,187095.313043,187095.313043,187095.313043,187095.313043,187095.313043,187095.313043


In [33]:
meses = contabilidad.columns.drop(['ceco', 'cuenta']).tolist()
meses

['julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

In [35]:
# Identificar el último mes con valor distinto de 0 o nan para cada fila

def identificar_ultimo_mes(df, meses):
    def ultimo_mes_fila(fila):
        for mes in reversed(meses):
            if pd.notna(fila[mes]) and fila[mes] != 0:
                return mes
        return None

    df['ultimo_mes'] = df.apply(ultimo_mes_fila, axis=1)
    return df

contabilidad = identificar_ultimo_mes(contabilidad, meses)
contabilidad

,ceco,cuenta,julio,agosto,septiembre,octubre,noviembre,diciembre,ultimo_mes
0,CO10A00000,5160350001,4519500.326,4519500.326,4519500.326,4519500.326,4519500.326,4519500.326,diciembre
1,CO10A00001,5160200001,322324,322324,322324,322324,322324,322324,diciembre
2,CO10A00003,5160200001,688144,688144,688144,688144,688144,688144,diciembre
3,CO10A00004,5160200001,526835,526835,526835,526835,526835,526835,diciembre
4,CO10A10000,5120750002,64500,64500,0,0,0,0,agosto
...,...,...,...,...,...,...,...,...,...
368,CO10F52000,5260200001,465253,465253,465253,465253,465253,465253,diciembre
369,CO10F60100,5220750002,3026233,3026233,3026233,3026233,3026233,3026233,diciembre
370,CO10F60100,5260200001,804809,804809,804809,618784,618784,618784,diciembre
371,CO10G00001,5260100001,187095.313043,187095.313043,187095.313043,187095.313043,187095.313043,187095.313043,diciembre
